In [1]:
# Example Application of Hough Line Transform
# Michael Timbes

# Hough Transform (Lines)
> The Hough transform can be used after applying an edge filter to find lines and circles in an image. The idea behind this method is taking a set of points produced by the filter and treating them as if they were all lines in another space (Hough space). In the space of lines, the points of intersections represent a line in the original space(cartesian space). 
## How to Get Lines
> These intersections are accounted for by votes which are stored in an accumulator. The accumulator has a certain structure to maintain most of the information about the point. <br>
<b>Accumulator:</b><br>
[ $\rho$, $\theta$ ] = <b>Vote</b> <br>
> As the accumulator structure suggests, the Hough space is in terms of a radius length and position (given by theta). The more votes a certain radius and position has, the greater the likelyhood of a line being present. Once the votes have been accounted for, one may rebuild the lines to plot ontop of the image. The relationship between rho and the (X,Y) values are outlined below in the polar representation of the cartesian space.<br>
$$\rho = x\cos (\theta) + y\sin(\theta)$$ 
The votes are based on the specific (X,Y) points. So that when rho is found from above, the votes at that specific rho and theta are added up. <br>
## Finding Lines from the Accumulator
> So with the accumulator, the first value of importance are the votes at each rho and theta. If someone is looking for just one line, it may be sufficient to find the point of the highest votes and then use the theta and rho value to plot the line. <br>
<b>Equation to Rebuild Lines</b> <br>
$m = \frac{-\cos(\theta)}{\sin(\theta)},b = \frac{\rho}{\sin(\theta)}$ <br>
$y = mx + b$ <br>
The idea is to plot the strongest line using the rho and theta associated with that value along all of the x axis.

In [2]:
from PIL import Image as img # Used to import photos
from PIL import ImageFilter as imfil # Filters
import matplotlib.pyplot as plt # Show images
import numpy as np # Math Ops

In [3]:
def find_refs(r, alpha):
    return (r*np.sin(alpha)), (r*np.cos(alpha))

In [ ]:
def gen_line(rho, theta, x):
    b = rho/(np.sin(theta))
    m = (-np.cos(theta)/sin(theta))
    y = m * x + b

In [4]:
def hough_trans(im):
    """
    INPUT:
    ((bin) image 2D vector)
    
    OUPUT:
    ((float64) theta): Angle vals in radians for precision
    ((float64) rhosp): Radius of line segments
    ((int) accu): Accumulator for tracking voting
    """
    # Assign References
    width, height = im.shape
    #print("Width, Height",width, height)
    
    # Max Line Segment Length
    max_lin = np.ceil(np.sqrt(width * width + height * height))
    max_lin = max_lin.astype(np.uint64)
    
    # Rho Space Vector
    rhosp = np.linspace(-max_lin, max_lin, max_lin * 2.0)
    
    # Theta Space Vector in Radians
    theta = np.deg2rad(np.arange(-90.0, 90.0, 1))
    
    # Build Trig Funcs
    cos_arr = np.cos(theta)
    cos_arr = cos_arr.astype(np.int64)
    sin_arr = np.sin(theta)
    sin_arr = sin_arr.astype(np.int64)
    
    steps = len(theta)
    
    # Build Accumulator
    R = 2*max_lin
    R = R.astype(np.uint64) 
    accu = np.zeros((R, steps))
    
    # Index Edge Points
    [idx_y, idx_x] = np.nonzero(im)
    #print(len(idx_y),len(idx_x))
    # Voting Process
    for i in range(len(idx_x)):
        x = idx_x[i]
        y = idx_y[i]
        for t in range(steps):
            rho = np.rint((x*cos_arr[t] + y*sin_arr[t]) + max_lin)
            rho = rho.astype(np.uint64)
            accu[rho, t] +=1
    return accu,theta,rhosp
        

In [5]:
stp_sign = img.open("stopsign.jpg", 'r')
stp_sign = stp_sign.convert("L")

# Smoothing Filter- add if there is a lot of noise.
#stp_sign = stp_sign.filter(imfil.GaussianBlur(2))

# Interesting that running a negative scale and then a positive scale filter and blending the two results 
# creates a very clear image of the edges for this stop sign. 

# Run Negative Scale Edge Filter
sobel_scale = .75
sobel_kernel_nX = stp_sign.filter(imfil.Kernel((3,3),[-1, 0, 1, -2, 0, 2, -1, 0, 1],-sobel_scale)) # X Direction
sobel_kernel_nY = stp_sign.filter(imfil.Kernel((3,3),[1, 2, 1, 0, 0, 0, -1, -2, -1],-sobel_scale)) # Y Direction

# Run Positive Scale Edge Filter
sobel_kernel_X = stp_sign.filter(imfil.Kernel((3,3),[-1, 0, 1, -2, 0, 2, -1, 0, 1],sobel_scale)) # X Direction
sobel_kernel_Y = stp_sign.filter(imfil.Kernel((3,3),[1, 2, 1, 0, 0, 0, -1, -2, -1],sobel_scale)) # Y Direction

# Negative Blending
neg_sign = img.blend(sobel_kernel_nX,sobel_kernel_nY,.5)
#neg_sign.show()

# Positive Blending
psv_sign = img.blend(sobel_kernel_X,sobel_kernel_Y,.5)
#psv_sign.show()

# Result of Negative and Positive
new_sign = img.blend(neg_sign,psv_sign,.5)
#new_sign.show()

# Cast to Numpy Array
stp_sign = np.array(new_sign)

# Generate Lines
ac, thet, rhos = hough_trans(stp_sign)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:20: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


68775 68775


In [11]:
# Information on Structure
print(ac.shape)
print("Number of Bins: ",ac.shape[0])
print("Degrees: ",ac.shape[1])

# Finding a Max
idx = np.argmax(ac)
print("Max Indx: ", idx)
rho = rhos[int(idx / ac.shape[0])]
theta = thet[idx % ac.shape[1]]
print(rho, np.rad2deg(theta))
Y,X = find_refs(rho,theta)


(786, 180)
Number of Bins:  786
Degrees:  180
Max Indx:  70741
-302.885350318 -89.0
302.839219413 -5.28607823766
